<a href="https://colab.research.google.com/github/ayaanzhaque/SuiSense/blob/master/notebooks/bert/tensorflow_bert_before.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.0
!pip install tensorflow_hub
!pip install bert-for-tf2
!pip install sentencepiece

ERROR: Could not find a version that satisfies the requirement tensorflow==2.0 (from versions: 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0)
ERROR: No matching distribution found for tensorflow==2.0


In [2]:
import tensorflow_hub as hub
import tensorflow as tf
import bert
FullTokenizer = bert.bert_tokenization.FullTokenizer
from tensorflow.keras.models import Model       # Keras is the new high level API for TensorFlow
import math

In [6]:
from tensorflow import keras
final_model = keras.models.load_model('baseline_model.h5')

In [7]:
max_seq_length = 512  # Your choice here.
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=[pooled_output, sequence_output])

# See BERT paper: https://arxiv.org/pdf/1810.04805.pdf
# And BERT implementation convert_single_example() at https://github.com/google-research/bert/blob/master/run_classifier.py

def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [8]:
s = "I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die I want to die"

stokens = tokenizer.tokenize(s)

stokens = ["[CLS]"] + stokens + ["[SEP]"]

input_ids = get_ids(stokens, tokenizer, max_seq_length)
input_masks = get_masks(stokens, max_seq_length)
input_segments = get_segments(stokens, max_seq_length)

pool_embs, all_embs = model.predict([[input_ids],[input_masks],[input_segments]])


print(pool_embs.shape)

ValueError: in user code:

    File "c:\Users\91897\Desktop\Suicide-Prediction-App\ayushi\lib\site-packages\keras\engine\training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\91897\Desktop\Suicide-Prediction-App\ayushi\lib\site-packages\keras\engine\training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\91897\Desktop\Suicide-Prediction-App\ayushi\lib\site-packages\keras\engine\training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\91897\Desktop\Suicide-Prediction-App\ayushi\lib\site-packages\keras\engine\training.py", line 2111, in predict_step
        return self(x, training=False)
    File "c:\Users\91897\Desktop\Suicide-Prediction-App\ayushi\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\91897\Des

In [23]:
predictions = final_model.predict(pool_embs)
print(predictions)

[[0.5506451]]
